In [119]:
import pandas as pd 
import duckdb
import matplotlib.pyplot as plt
import numpy as np
from typing import Any 

In [2]:
conn = duckdb.connect('../data/workout_db.db')

In [3]:
conn.sql("SHOW TABLES")

┌──────────────────┐
│       name       │
│     varchar      │
├──────────────────┤
│ raw_landmarks    │
│ workout          │
│ workout_analysis │
└──────────────────┘

### Get data 

In [116]:
def gey_id_frame()->dict[str,int]| None:
    try:
        query_id:str = "Select max(id) from workout"
        res_id:tuple|None = conn.sql(query_id).fetchone()
        if res_id is None:
            raise ValueError("No id found in database.")
        else:
            workout_id:int = res_id[0]
        # get the first good form and backtrack 10 frames as the start frame (the idea is to avoid analysing the frames when user is setting up)
        strat_frame_query:str = f" SELECT  frame from workout_analysis  where workout_id ={workout_id} and SUBSTRING(form_issues,1,4)= 'Good' order by frame"
        res_start_frame:tuple|None= conn.sql(strat_frame_query).fetchone() 
        if res_start_frame is None:
            raise ValueError("No analysis data found for the last workout.")
        else:
            frame_start:int = res_start_frame[0]-10
    except Exception as e:
        print(f"Error retrieving data: {e}")
        return None
    return {'id': workout_id, 'frame_start': frame_start}
    
    

In [100]:
id_frame = gey_id_frame()
id_used = id_frame['id']
frame_start = id_frame['frame_start']
id_used, frame_start

(1, 36)

In [330]:
def extract_metadata()-> pd.Series | None:

    try:
        metadata:pd.DataFrame = conn.sql(f"SELECT * FROM workout where id = {id_used}").df()
        if metadata is None:
            raise ValueError("No metadata found for the last workout.")
        end_time_query = f"SELECT max(timestamp) FROM workout_analysis where workout_id = {id_used}"
        end_time:tuple|None = conn.sql(end_time_query).fetchone()
        if end_time is None:
            raise ValueError("No end time found for the last workout.")
        end_time = end_time[0]
        metadata['end_time'] = end_time
        metadata['duration']= metadata.apply(lambda x: round((x['end_time']-x['timestamp_start']).total_seconds(),2),axis=1)
    except Exception as e:
        print(f"Error retrieving metadata: {e}")
        return None
    
    return metadata.iloc[0]

In [331]:
metadata = extract_metadata()

In [332]:
metadata

ID                                                                       1
workout_name                                                      push-ups
timestamp_start                                 2025-06-20 12:31:34.334510
rep_goal                                                                10
strictness_crit                                                      loose
strictness_definition                                                 15.0
left_side                                                             True
ldmrks_of_interest       {'shoulder': 11, 'elbow': 13, 'wrist': 15, 'hi...
end_time                                        2025-06-20 12:32:33.166122
duration                                                             58.83
Name: 0, dtype: object

In [276]:
id_used

1

In [341]:
def extract_analysis(metadata)-> dict[str,Any] | None:
    try: 
        # get total number of reps 
        tot_reps_query:str = f"select max(rep_count) from workout_analysis where workout_id = {id_used} and frame >= {frame_start}"
        total_reps_res:tuple|None = conn.sql(tot_reps_query).fetchone()
        if total_reps_res is None:
            raise ValueError("No analysis data found for current workout")
        else:
            total_reps:int = total_reps_res[0]
        
        # duration of each repetitons 
        frames_rep_query:str = f"""select min(frame) as start_frame,max(frame) as end_frame,rep_count+1 as repetitions, round(epoch(max(timestamp) - min(timestamp)),1) as duration ,
                    from workout_analysis where workout_id = {id_used} and frame>= {frame_start} and repetitions <= {metadata['rep_goal']} group by rep_count"""
        frames_rep:pd.DataFrame = conn.sql(frames_rep_query).df()
        
        # average time rep 
        avg_duration:float = round(frames_rep['duration'].mean(),2)
        
        # form issues
        form_issues_query:str = f"""SELECT form_issues, count(*) as count FROM workout_analysis WHERE workout_id = {id_used} AND frame >= {frame_start} GROUP BY form_issues"""
        form_issues:list = conn.sql(form_issues_query).fetchall()
        
        frames_issues_quesry:str = f"""SELECT  form_issues, LIST(frame) FROM workout_analysis WHERE workout_id = {id_used} AND frame >= {frame_start} and form_issues not like 'Good form' group by form_issues  """
        frames_w_issues:list = conn.sql(frames_issues_quesry).fetchall()
        
        worst_reps_query:str = f"""SELECT rep_count+1 as repetitions, count(form_issues) as count
         FROM workout_analysis 
         WHERE workout_id = {id_used} AND frame >= {frame_start} and 
         form_issues not like 'Good form'
         GROUP BY repetitions
         ORDER BY count DESC
         LIMIT 3"""
        worst_reps:list = conn.sql(worst_reps_query).fetchall()
        
        # exctract angles data 
        dif_angles_query:str = f""" select list(distinct(angle_element.name)) as angle_names FROM workout_analysis, UNNEST(angles_data) as t(angle_element)"""
        diff_names_ang_res:tuple|None = conn.sql(dif_angles_query).fetchone()
        if diff_names_ang_res is None:
            raise ValueError("Error exctracting names of angles")
        else:
            different_names:list = diff_names_ang_res[0]
        
        cases:list[str] = [f"""MAX(case WHEN element.name ='{name}' THEN round(element.value,2) end) as {name}_angle,
        MAX(case WHEN element.name ='{name}' THEN element.joint_indices end) as {name}_indices
        """ for name in different_names]
        angles_query:str= f"""select frame, {','.join(cases)} from workout_analysis, unnest(angles_data) as t(element) where workout_id = {id_used} AND frame >= {frame_start}
        and form_issues not like 'Good form' group by frame"""
        angles_df:pd.DataFrame = conn.sql(angles_query).df()

        index_to_name_map:dict = {v: k for k, v in metadata['ldmrks_of_interest'].items()} #to be removed
        
        # Define the list of columns to transform
        indices_col:list = [f"{name}_indices" for name in different_names]

        transformed_cols = {
            col: angles_df[col].apply(lambda idx_list: [index_to_name_map.get(k) for k in idx_list])
            for col in indices_col }

        angles_final = angles_df.assign(**transformed_cols)
    
        all_angles_query:str = f"""select frame, {','.join(cases)} from workout_analysis, unnest(angles_data) as t(element) where workout_id = {id_used} AND frame >= {frame_start}
        group by frame"""
        angles_desc = conn.sql(all_angles_query).df().describe()
        
    except Exception as e:
        print(f"Error extracting analysis data: {e}")
        return None
    extracted_data = {
        'total_repetitions': total_reps,
        'average_time_repetitions':avg_duration,
        'form_issues':form_issues,
        'frames_with_issues': frames_w_issues,
        'worst_reps':worst_reps,
        'frames_reps':frames_rep,
        'angles_data':angles_final,
        'angles_desc_stats':angles_desc
    }
    return extracted_data


In [342]:
extract_analysis(metadata)

{'total_repetitions': 10,
 'average_time_repetitions': 3.98,
 'form_issues': [('shoulders not aligned with wrists', 3),
  ('Good form', 314),
  ('body not straight', 91)],
 'frames_with_issues': [('shoulders not aligned with wrists', [203, 252, 253]),
  ('body not straight',
   [36,
    37,
    38,
    39,
    40,
    41,
    42,
    43,
    44,
    45,
    73,
    75,
    91,
    92,
    93,
    94,
    95,
    96,
    97,
    98,
    99,
    100,
    101,
    102,
    103,
    104,
    105,
    106,
    127,
    128,
    129,
    130,
    131,
    132,
    133,
    134,
    135,
    136,
    137,
    168,
    169,
    170,
    171,
    173,
    178,
    180,
    181,
    182,
    183,
    205,
    206,
    207,
    208,
    209,
    210,
    211,
    212,
    213,
    281,
    282,
    283,
    284,
    285,
    306,
    308,
    309,
    310,
    311,
    314,
    315,
    316,
    317,
    318,
    319,
    320,
    321,
    322,
    323,
    324,
    325,
    326,
    327,
    328

In [343]:
def extract_raw_landmarks(metadata)->dict[str,pd.DataFrame]|None:
    try:
        #get raw landmarks only for landmarks of interest
        index_to_name_map:dict = {v: k for k, v in metadata['ldmrks_of_interest'].items()} # to be removed
        ofinterest_keys = ''
        ofinterest_keys = ','.join([f""" round(landmark_{k}_x,2) as {v}_x, round(landmark_{k}_y,2) as {v}_y""" for k,v in index_to_name_map.items()])
        of_interest_query: str = f""" select frame, {ofinterest_keys} from raw_landmarks where frame >= {frame_start} and workout_id == {id_used}"""
        raw_of_int:pd.DataFrame = conn.sql(of_interest_query).df()
        of_int_desc:pd.DataFrame = raw_of_int.describe()
    except Exception as e:
        print(f"Error extracting analysis data: {e}")
        return None
    output = {
        'raw_of_interest':raw_of_int,
        'of_interest_desc':of_int_desc
    }
    return output


In [344]:
extract_raw_landmarks(metadata)

{'raw_of_interest':      frame  shoulder_x  shoulder_y  elbow_x  elbow_y  wrist_x  wrist_y  hip_x  \
 0       36        0.27        0.28     0.33     0.47     0.34     0.67   0.55   
 1       37        0.27        0.28     0.33     0.47     0.34     0.66   0.53   
 2       38        0.27        0.27     0.33     0.47     0.34     0.66   0.53   
 3       39        0.27        0.26     0.34     0.45     0.34     0.66   0.54   
 4       40        0.27        0.26     0.34     0.45     0.34     0.65   0.53   
 ..     ...         ...         ...      ...      ...      ...      ...    ...   
 403    439        0.21        0.38     0.33     0.48     0.33     0.66   0.48   
 404    440        0.22        0.35     0.33     0.47     0.33     0.66   0.48   
 405    441        0.22        0.33     0.33     0.47     0.33     0.66   0.48   
 406    442        0.23        0.31     0.33     0.47     0.33     0.66   0.49   
 407    443        0.24        0.29     0.33     0.46     0.33     0.66   0.50 

In [345]:
analysis = extract_analysis(metadata)


In [346]:
analysis.items()

dict_items([('total_repetitions', 10), ('average_time_repetitions', 3.98), ('form_issues', [('Good form', 314), ('body not straight', 91), ('shoulders not aligned with wrists', 3)]), ('frames_with_issues', [('shoulders not aligned with wrists', [203, 252, 253]), ('body not straight', [36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 73, 75, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 168, 169, 170, 171, 173, 178, 180, 181, 182, 183, 205, 206, 207, 208, 209, 210, 211, 212, 213, 281, 282, 283, 284, 285, 306, 308, 309, 310, 311, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 367, 371, 379, 381, 382, 387, 388, 420])]), ('worst_reps', [(8, 20), (2, 16), (1, 12)]), ('frames_reps',    start_frame  end_frame  repetitions  duration
0           36         89            1       5.4
1           90        123            2       3.3
2          124        165            3       4.1
3          166     

In [ ]:
from langchain_core.prompts import PromptTemplate

def create_prompt_template(metadata,analysis,raw_ldmrks)->str:
    
    template = """ 
    [ROLE]
    Act as an expert, data-driven fitness coach. Your tone should be encouraging, clear, and direct.

    [CONTEXT & GOAL]
    You are analyzing a user's workout based on the following data. The user's goal was to {user_goal}.

    """
    
    
    
    metadata_string = f"""Type of workout: {metadata['workout_name']},
    start_time: {metadata['timestamp_start']},
    end_time: {metadata['end_time']},
    duration:{metadata['duration']} seconds,
    number of repetitions: {metadata['rep_goal']},
    strictness: {metadata['strictness_crit']}, meaning the allowed deviation from the ideal form is {metadata['strictness_definition']},
    the landmarks of interest are: {metadata['ldmrks_of_interest'].values()} the relate to the following body parts: {metadata['ldmrks_of_interest'].values()}
    """
    
    form_issues = analysis['form_issues']
    frames_w_issues = analysis['frames_with_issues']
    frames_rep = analysis['frames_reps']
    worst_reps = analysis['worst_reps']
    angles_issues = analysis['angles_data']
    angles_stats = analysis['angles_desc_stats']
    


    return ''

In [91]:
id_used = conn.sql("Select max(id) from workout").fetchone()[0]

In [92]:
id_used

1

In [6]:
conn.sql(f"select * from workout where id ={id_used}")

┌───────┬──────────────┬───────────────────────────┬──────────┬─────────────────┬───────────────────────┬───────────┬───────────────────────────────────────────────────────────────────────┐
│  ID   │ workout_name │      timestamp_start      │ rep_goal │ strictness_crit │ strictness_definition │ left_side │                          ldmrks_of_interest                           │
│ int32 │   varchar    │         timestamp         │  int32   │     varchar     │        double         │  boolean  │                         map(varchar, integer)                         │
├───────┼──────────────┼───────────────────────────┼──────────┼─────────────────┼───────────────────────┼───────────┼───────────────────────────────────────────────────────────────────────┤
│     1 │ push-ups     │ 2025-06-20 12:31:34.33451 │       10 │ loose           │                  15.0 │ true      │ {shoulder=11, elbow=13, wrist=15, hip=23, ankle=27, knee=25, toes=31} │
└───────┴──────────────┴──────────────────────────

In [7]:
conn.sql(f"SELECT * FROM workout_analysis where workout_id = {id_used}")

┌────────────┬───────┬────────────────────────────┬───────────────┬───────────┬─────────┬───────────────────────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ workout_id │ frame │         timestamp          │ series_number │ rep_count │  down   │            form_issues            │                                                                        angles_data                                                                        │
│   int32    │ int32 │         timestamp          │     int32     │   int32   │ boolean │              varchar              │                                             struct("name" varchar, "value" double, joint_indices integer[])[]                                             │
├────────────┼───────┼────────────────────────────┼───────────────┼───────────┼─────────┼───────────────────────────────────┼─────────────────────────────

In [ ]:
# find first good form 

frame_start = conn.sql(f" SELECT  frame from workout_analysis  where workout_id ={id_used} and SUBSTRING(form_issues,1,4)= 'Good' order by frame").fetchone()[0] -10

36

### Features to extract from the data: 
#### Metdata:
- inlcude everything from the metadata file..
#### Workout analysis:
- Total number of reps. 
- Average time per rep. 
- Total counts of different forms
- Counts of form aggregated per rep.
- Time at lowest point of each rep.
- Time at highest point of each rep.
- Average time at lowest point of each set.
- reps with highest number of form errors.
- Angles distribution, std, min, max, mean, median.
- start and end frame of each rep.


#### raw landmarks:
- Only landmarks of interest. 
- Time at lowest point of each rep.
- Time at highest point of each rep.
- Average time at lowest point of each set.


### Metdata extraction

In [16]:
metadata = conn.sql(f"SELECT * FROM workout where id = {id_used}").df().iloc[0]
metadata

ID                                                                       1
workout_name                                                      push-ups
timestamp_start                                 2025-06-20 12:31:34.334510
rep_goal                                                                10
strictness_crit                                                      loose
strictness_definition                                                 15.0
left_side                                                             True
ldmrks_of_interest       {'shoulder': 11, 'elbow': 13, 'wrist': 15, 'hi...
Name: 0, dtype: object

In [40]:
end_time = conn.sql(f"SELECT max(timestamp) FROM workout_analysis where workout_id = {id_used}").fetchone()[0]
landmarks_of_interest = metadata['ldmrks_of_interest']
end_time

datetime.datetime(2025, 6, 20, 12, 32, 33, 166122)

In [18]:
metadata_string = f"""Type of workout: {metadata['workout_name']},
 start_time: {metadata['timestamp_start']},
 end_time: {end_time},
 duration:{round((end_time-metadata['timestamp_start']).total_seconds(),2)} seconds,
 number of repetitions: {metadata['rep_goal']},
 strictness: {metadata['strictness_crit']}, meaning the allowed deviation from the ideal form is {metadata['strictness_definition']},
 the landmarks of interest are: {metadata['ldmrks_of_interest'].values()} the relate to the following body parts: {metadata['ldmrks_of_interest'].values()}
"""
metadata_string

'Type of workout: push-ups,\n start_time: 2025-06-20 12:31:34.334510,\n end_time: 2025-06-20 12:32:33.166122,\n duration:58.83 seconds,\n number of repetitions: 10,\n strictness: loose, meaning the allowed deviation from the ideal form is 15.0,\n the landmarks of interest are: dict_values([11, 13, 15, 23, 27, 25, 31]) the relate to the following body parts: dict_values([11, 13, 15, 23, 27, 25, 31])\n'

#### workout analysis
- Total number of reps. 
- Average time per rep.
- start and end frame of each rep. 
- Total counts of different forms
- Counts of form aggregated per rep.
- reps with highest number of form errors.
- Angles distribution, std, min, max, mean, median.

In [19]:
# Total number of repetitions
total_reps = conn.sql(f"select max(rep_count) from workout_analysis where workout_id = {id_used} and frame >= {frame_start}").fetchone()[0]
total_reps

10

In [131]:
# duration of each repetitons 
frames_rep = conn.sql(f"""select min(frame) as start_frame,max(frame),min(timestamp) as start_t,max(timestamp) as end_t,rep_count+1 as repetitions,
                        from workout_analysis where workout_id = {id_used}  and frame>= {frame_start}  group by rep_count""").df()
frames_rep

,start_frame,max(frame),start_t,end_t,repetitions
0,36,89,2025-06-20 12:31:52.250442,2025-06-20 12:31:57.622131,1
1,90,123,2025-06-20 12:31:57.677578,2025-06-20 12:32:00.994371,2
2,124,165,2025-06-20 12:32:01.097842,2025-06-20 12:32:05.211785,3
3,166,201,2025-06-20 12:32:05.324840,2025-06-20 12:32:08.830460,4
4,202,250,2025-06-20 12:32:08.929739,2025-06-20 12:32:13.758503,5
5,251,273,2025-06-20 12:32:13.860591,2025-06-20 12:32:16.046992,6
6,274,303,2025-06-20 12:32:16.146933,2025-06-20 12:32:19.077231,7
7,304,362,2025-06-20 12:32:19.185085,2025-06-20 12:32:25.017154,8
8,363,405,2025-06-20 12:32:25.112356,2025-06-20 12:32:29.339515,9
9,406,442,2025-06-20 12:32:29.442225,2025-06-20 12:32:33.065203,10


In [134]:
# duration of each repetitons 
frames_rep = conn.sql(f"""select min(frame) as start_frame,max(frame) as end_frame,rep_count+1 as repetitions, round(epoch(max(timestamp) - min(timestamp)),1) as duration ,
                    from workout_analysis where workout_id = {id_used}  and frame>= {frame_start} and repetitions <= {metadata['rep_goal']} group by rep_count""").df()
frames_rep

,start_frame,end_frame,repetitions,duration
0,36,89,1,5.4
1,90,123,2,3.3
2,124,165,3,4.1
3,166,201,4,3.5
4,202,250,5,4.8
5,251,273,6,2.2
6,274,303,7,2.9
7,304,362,8,5.8
8,363,405,9,4.2
9,406,442,10,3.6


In [121]:
frames_rep['duration'] = round((frames_rep['end_t'] - frames_rep['start_t']).dt.total_seconds(),1)
frames_rep = frames_rep[frames_rep['repetitions']<= metadata['rep_goal']]

In [122]:
frames_rep

,min(frame),max(frame),start_t,end_t,repetitions,duration
0,36,89,2025-06-20 12:31:52.250442,2025-06-20 12:31:57.622131,1,5.4
1,90,123,2025-06-20 12:31:57.677578,2025-06-20 12:32:00.994371,2,3.3
2,124,165,2025-06-20 12:32:01.097842,2025-06-20 12:32:05.211785,3,4.1
3,166,201,2025-06-20 12:32:05.324840,2025-06-20 12:32:08.830460,4,3.5
4,202,250,2025-06-20 12:32:08.929739,2025-06-20 12:32:13.758503,5,4.8
5,251,273,2025-06-20 12:32:13.860591,2025-06-20 12:32:16.046992,6,2.2
6,274,303,2025-06-20 12:32:16.146933,2025-06-20 12:32:19.077231,7,2.9
7,304,362,2025-06-20 12:32:19.185085,2025-06-20 12:32:25.017154,8,5.8
8,363,405,2025-06-20 12:32:25.112356,2025-06-20 12:32:29.339515,9,4.2
9,406,442,2025-06-20 12:32:29.442225,2025-06-20 12:32:33.065203,10,3.6


In [22]:
avg_duration = frames_rep['duration'].mean()
avg_duration

3.9799999999999995

In [23]:
fps = np.median([(frames_rep['max(frame)'].iloc[i] - frames_rep['min(frame)'].iloc[i])/frames_rep['duration'].iloc[i] for i in range(len(frames_rep))])
fps


10.0

In [241]:
# form issues
form_issues = conn.sql(f"""SELECT form_issues, count(*) as count FROM workout_analysis WHERE workout_id = {id_used} AND frame >= {frame_start} GROUP BY form_issues""").fetchall()
form_issues

[('Good form', 314),
 ('body not straight', 91),
 ('shoulders not aligned with wrists', 3)]

In [239]:
frames_w_issues = conn.sql(f"""SELECT  form_issues, LIST(frame) FROM workout_analysis WHERE workout_id = {id_used} AND frame >= {frame_start} and form_issues not like 'Good form' group by form_issues  """).fetchall()


In [240]:
frames_w_issues[0]

('shoulders not aligned with wrists', [203, 252, 253])

In [26]:
worst_reps = conn.sql(f"""SELECT rep_count+1 as repetitions, count(form_issues) as count
         FROM workout_analysis 
         WHERE workout_id = {id_used} AND frame >= {frame_start} and 
         form_issues not like 'Good form'
         GROUP BY repetitions
         ORDER BY count DESC
         LIMIT 3""").df()

In [193]:
query = f""" select list(distinct(angle_element.name)) as angle_names FROM workout_analysis, UNNEST(angles_data) as t(angle_element)"""

different_names = conn.sql(query).fetchone()[0]

different_names

['body', 'elbow']

In [ ]:
[f"""
 select frame
 
 as {name}_angle, as {name}_joints""" for name in different_names]

['as body_angle, ... as body_joints', 'as elbow_angle, ... as elbow_joints']

In [245]:
cases = [f"""MAX(case WHEN element.name ='{name}' THEN round(element.value,2) end) as {name}_angle,
MAX(case WHEN element.name ='{name}' THEN element.joint_indices end) as {name}_indices
""" for name in different_names]

angles_query= f"""select frame, {','.join(cases)} from workout_analysis, unnest(angles_data) as t(element) where workout_id = {id_used} AND frame >= {frame_start}
 and form_issues not like 'Good form'group by frame"""


In [250]:
angles = conn.sql(angles_query).df()

In [318]:
index_to_name_map:dict = {v: k for k, v in metadata['ldmrks_of_interest'].items()}
        
# Define the list of columns to transform
indices_col:list = [f"{name}_indices" for name in different_names]

# The vectorized operation using .apply()
# We define a lambda function that takes a list `idx_list` and performs the lookup for each item in it.
# .apply() runs this lambda function on every cell in the selected DataFrame slice.
transformed_cols = {
    col: angles[col].apply(lambda idx_list: [index_to_name_map.get(k) for k in idx_list])
    for col in indices_col
}

final_df = angles.assign(**transformed_cols)

final_df

,frame,body_angle,body_indices,elbow_angle,elbow_indices
0,36,124.42,"[shoulder, hip, ankle]",165.36,"[wrist, elbow, shoulder]"
1,37,125.16,"[shoulder, hip, ankle]",166.45,"[wrist, elbow, shoulder]"
2,38,127.05,"[shoulder, hip, ankle]",164.34,"[wrist, elbow, shoulder]"
3,39,135.72,"[shoulder, hip, ankle]",163.66,"[wrist, elbow, shoulder]"
4,40,127.27,"[shoulder, hip, ankle]",164.09,"[wrist, elbow, shoulder]"
...,...,...,...,...,...
89,381,157.85,"[shoulder, hip, ankle]",166.75,"[wrist, elbow, shoulder]"
90,382,159.75,"[shoulder, hip, ankle]",166.58,"[wrist, elbow, shoulder]"
91,387,157.33,"[shoulder, hip, ankle]",163.90,"[wrist, elbow, shoulder]"
92,388,159.76,"[shoulder, hip, ankle]",155.68,"[wrist, elbow, shoulder]"


In [270]:
indices

{'body_indices': ['shoulder', 'hip', 'ankle'],
 'elbow_indices': ['wrist', 'elbow', 'shoulder']}

In [33]:
angles.groupby('name').describe().T

name               body       elbow
value count  408.000000  408.000000
      mean   165.349926  136.835564
      std      8.942225   40.133215
      min    124.420000   49.820000
      25%    161.017500  102.257500
      50%    165.885000  162.750000
      75%    171.917500  167.370000
      max    180.000000  171.380000

In [34]:
indices = angles.groupby('name')['joint_indices'].apply(lambda x: x.iloc[0]).reset_index()

In [41]:
loi:dict = {k:v for v,k in landmarks_of_interest.items()}

In [42]:
indices['indices_meaning']= indices.apply(lambda x: [loi.get(k) for k in x['joint_indices']],axis=1)

In [43]:
indices

,name,joint_indices,indices_meaning
0,body,"[11, 23, 27]","[shoulder, hip, ankle]"
1,elbow,"[15, 13, 11]","[wrist, elbow, shoulder]"


#### raw landmarks
- Only landmarks of interest. 
- Time at lowest point of each rep.
- Time at highest point of each rep.
- Average time at lowest point of each set.

In [44]:
raw_landmarks = conn.sql(f"""SELECT * FROM raw_landmarks WHERE frame >= {frame_start} and workout_id = {id_used}""")
raw_landmarks

┌───────┬────────────────────────────┬────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬────────────────────┬─────────────────────┬────────────────────┬─────────────────────┬────────────────────┬─────────────────────┬────────────────────┬─────────────────────┬────────────────────┬─────────────────────┬────────────────────┬─────────────────────┬─────────

In [45]:
ofinterest_keys = ''
ofinterest_keys = ','.join([f""" round(landmark_{k}_x,2) as {v}_x, round(landmark_{k}_y,2) as {v}_y""" for k,v in loi.items()])
ofinterest_keys

' round(landmark_11_x,2) as shoulder_x, round(landmark_11_y,2) as shoulder_y, round(landmark_13_x,2) as elbow_x, round(landmark_13_y,2) as elbow_y, round(landmark_15_x,2) as wrist_x, round(landmark_15_y,2) as wrist_y, round(landmark_23_x,2) as hip_x, round(landmark_23_y,2) as hip_y, round(landmark_27_x,2) as ankle_x, round(landmark_27_y,2) as ankle_y, round(landmark_25_x,2) as knee_x, round(landmark_25_y,2) as knee_y, round(landmark_31_x,2) as toes_x, round(landmark_31_y,2) as toes_y'

In [46]:
loi_raw = conn.sql(f""" select frame, {ofinterest_keys} 
                   from raw_landmarks where frame >= {frame_start} and workout_id == {id_used}""").df()
loi_raw

,frame,shoulder_x,shoulder_y,elbow_x,elbow_y,wrist_x,wrist_y,hip_x,hip_y,ankle_x,ankle_y,knee_x,knee_y,toes_x,toes_y
0,36,0.27,0.28,0.33,0.47,0.34,0.67,0.55,0.27,0.92,0.56,0.71,0.50,0.94,0.67
1,37,0.27,0.28,0.33,0.47,0.34,0.66,0.53,0.27,0.89,0.54,0.70,0.52,0.93,0.65
2,38,0.27,0.27,0.33,0.47,0.34,0.66,0.53,0.27,0.90,0.54,0.70,0.50,0.94,0.65
3,39,0.27,0.26,0.34,0.45,0.34,0.66,0.54,0.27,0.91,0.55,0.71,0.46,0.94,0.66
4,40,0.27,0.26,0.34,0.45,0.34,0.65,0.53,0.30,0.87,0.55,0.67,0.53,0.92,0.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403,439,0.21,0.38,0.33,0.48,0.33,0.66,0.48,0.44,0.94,0.57,0.71,0.52,0.97,0.67
404,440,0.22,0.35,0.33,0.47,0.33,0.66,0.48,0.43,0.94,0.58,0.71,0.52,0.97,0.67
405,441,0.22,0.33,0.33,0.47,0.33,0.66,0.48,0.41,0.94,0.58,0.70,0.52,0.96,0.67
406,442,0.23,0.31,0.33,0.47,0.33,0.66,0.49,0.39,0.94,0.58,0.70,0.52,0.96,0.67


In [47]:
loi_raw.describe()

,frame,shoulder_x,shoulder_y,elbow_x,elbow_y,wrist_x,wrist_y,hip_x,hip_y,ankle_x,ankle_y,knee_x,knee_y,toes_x,toes_y
count,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000
mean,239.500000,0.246348,0.343873,0.336078,0.473603,0.330809,0.661961,0.513284,0.407917,0.938775,0.566299,0.718554,0.520147,0.964877,0.666225
std,117.923704,0.018162,0.114234,0.022252,0.028293,0.003850,0.005346,0.015877,0.084262,0.009812,0.008992,0.009411,0.032767,0.014037,0.005778
min,36.000000,0.210000,0.240000,0.310000,0.440000,0.320000,0.650000,0.480000,0.270000,0.870000,0.540000,0.670000,0.460000,0.920000,0.650000
25%,137.750000,0.230000,0.260000,0.320000,0.450000,0.330000,0.660000,0.500000,0.340000,0.930000,0.560000,0.710000,0.500000,0.950000,0.660000
50%,239.500000,0.260000,0.270000,0.330000,0.460000,0.330000,0.660000,0.520000,0.370000,0.940000,0.570000,0.720000,0.510000,0.960000,0.670000
75%,341.250000,0.260000,0.440000,0.352500,0.500000,0.330000,0.660000,0.530000,0.480000,0.940000,0.570000,0.722500,0.550000,0.980000,0.670000
max,443.000000,0.270000,0.580000,0.390000,0.560000,0.340000,0.690000,0.550000,0.600000,0.960000,0.600000,0.740000,0.590000,0.990000,0.680000


In [58]:
frames_issues_details = loi_raw.merge(frames_w_issues, on='frame', how='right')
frames_issues_details

,frame,shoulder_x,shoulder_y,elbow_x,elbow_y,wrist_x,wrist_y,hip_x,hip_y,ankle_x,ankle_y,knee_x,knee_y,toes_x,toes_y,repetitions,form_issues
0,36,0.27,0.28,0.33,0.47,0.34,0.67,0.55,0.27,0.92,0.56,0.71,0.50,0.94,0.67,1.0,body not straight
1,37,0.27,0.28,0.33,0.47,0.34,0.66,0.53,0.27,0.89,0.54,0.70,0.52,0.93,0.65,1.0,body not straight
2,38,0.27,0.27,0.33,0.47,0.34,0.66,0.53,0.27,0.90,0.54,0.70,0.50,0.94,0.65,1.0,body not straight
3,39,0.27,0.26,0.34,0.45,0.34,0.66,0.54,0.27,0.91,0.55,0.71,0.46,0.94,0.66,1.0,body not straight
4,40,0.27,0.26,0.34,0.45,0.34,0.65,0.53,0.30,0.87,0.55,0.67,0.53,0.92,0.65,1.0,body not straight
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,381,0.26,0.26,0.32,0.45,0.33,0.66,0.53,0.34,0.94,0.56,0.72,0.50,0.95,0.66,9.0,body not straight
90,382,0.26,0.26,0.32,0.45,0.33,0.66,0.52,0.34,0.94,0.57,0.72,0.49,0.95,0.66,9.0,body not straight
91,387,0.26,0.27,0.32,0.46,0.33,0.66,0.52,0.35,0.94,0.57,0.72,0.51,0.95,0.66,9.0,body not straight
92,388,0.26,0.28,0.33,0.46,0.33,0.66,0.52,0.36,0.94,0.57,0.72,0.51,0.95,0.66,9.0,body not straight


In [ ]:
conn.close()